In [1]:
gradovi = open('gradovi.txt', 'r', encoding='UTF-8')

gradovi = gradovi.read()

gradovi = gradovi.split(' ')

while '' in gradovi:
    gradovi.remove('')


In [2]:
gradovi_dict = {}
grad = ''

for i, item in enumerate(gradovi):

    if i < 2:
        continue

    if not item[-1].isdigit():
        if gradovi[i-1].isdigit():
            sifra = gradovi[i-1]
        grad += item + ' '
    
    if not item[0].isdigit() and item[-1].isdigit():
        if grad != '':
            grad += item[:-3]

            gradovi_dict[grad] = [sifra]

            grad = ''
            sifra = 0
        
        else:
            gradovi_dict[item[:-3]] = [gradovi[i-1]]

for key, val in gradovi_dict.items():
    gradovi_dict[key.strip()] = gradovi_dict[key]

gradovi_dict['ZAGREB'] = gradovi_dict.pop('GRAD ZAGREB')


In [4]:
racuni = open('racuni.txt', 'r', encoding='UTF-8')

lst = []
racuni_dict = {}

for line in racuni:
    try:
        grad = " ".join(line.split()[:-1])
        iban = line.split()[-1]

        racuni_dict[grad] = iban

        lst.append([grad, iban])
    except:
        continue
prirez = open('stope.txt', 'r', encoding='UTF-8')

prirez_dict = {}

for line in prirez:
    try:
        grad = " ".join(line.split()[:-1])
        stopa = float(line.split()[-1])

        prirez_dict[grad.upper()] = stopa

        lst2.append([grad.upper(), stopa])
    except:
        continue

for grad, joppd in gradovi_dict.items():
    for i, item in enumerate(lst):
        if item[0] == grad:
            gradovi_dict[grad].append(item[1])
    
    if len(gradovi_dict[grad]) < 2:
        gradovi_dict[grad].append(None)
    
        





Problem kod Andrijaševci	8

Problem kod Antunovac	10

Problem kod Babina Greda	5

Problem kod Bale	1

Problem kod Barban	5

Problem kod Bedenica	3

Problem kod Bednja	10

Problem kod Beli Manastir	5

Problem kod Belica	0

Problem kod Belišće	7

Problem kod Benkovac	5

Problem kod Beretinec	7

Problem kod Bilice	10

Problem kod Bilje	5

Problem kod Biograd na Moru	12

Problem kod Biskupija	4

Problem kod Bistra	10

Problem kod Bizovac	3

Problem kod Bjelovar	0

Problem kod Blato	10

Problem kod Bol	10

Problem kod Borovo	6

Problem kod Bošnjaci	5

Problem kod Brckovljani	3

Problem kod Brdovec	10

Problem kod Brela	5

Problem kod Brestovac	5

Problem kod Breznica	10

Problem kod Breznički Hum	3

Problem kod Brinje	10

Problem kod Brodski Stupnik	10

Problem kod Buje	6

Problem kod Bukovlje	10

Problem kod Buzet	6

Problem kod Cerna	5

Problem kod Cernik	10

Problem kod Cerovlje	2

Problem kod Cestica	10

Problem kod Cista Provo	3

Problem kod Civljane	5

Problem kod Crikvenica	10

Probl

In [10]:
from pandas import DataFrame

In [11]:
df_cities = DataFrame.from_dict(gradovi_dict, orient='index')

df_cities.reset_index(inplace=True)
df_cities.rename(columns={'index': 'city_name', 0: 'pk', 1: 'IBAN', 2: 'stopa'}, inplace=True)

In [13]:
from bs4 import BeautifulSoup
from requests import get

banks_page = get('https://tockanai.hr/banke/')

soup = BeautifulSoup(banks_page.content, 'html.parser')

div = soup.find_all("div", attrs={"class": "vc_tta-panels-container"})



In [15]:
banks = {}

for item in div:

    bank = str(item.find_next('h3'))
    
    bank_name = bank[4:-5].strip()

    p = str(item.findChildren('p'))

    try:
        s = p.index('Poštanski broj i grad')
        e = p.index('<br/>', s+len('Poštanski broj i grad'))

        res = p[s+len('Poštanski broj i grad:')+7:e]

        if ', ' in p[s+len('Poštanski broj i grad:')+7:e]:   
            res = res.replace(',', '') 

        city = res.strip()

    except ValueError:
        city = ''


    try:
        s = p.index('Adresa')
        e = p.index('<br/>', s+7)

        address = p[s+7:e].strip()
        
    except ValueError:
        address = ''

    try:
        s = p.index('OIB')
        e = p.index('<br/>', s+4)

        oib = p[s+4:e].strip()

    except ValueError:
        oib = ''
    
    try:
        s = p.index('Matični broj')
        e = p.index('<br/>', s+len('Matični broj'))

        mib = p[s+len('Matični broj:'):e].strip()
  
    except ValueError:
        mib = ''

    try:
        s = p.index('IBAN')
        e = p.index('<br/>', s+len('IBAN'))

        iban = p[s+len('IBAN:'):e].strip()
  
    except ValueError:
        iban = ''
    
    banks[bank_name] = {'oib': oib, 'mib': mib, 'address': address, 'city': city, 'iban': iban}

for podaci in banks.values():
    podaci['city'] = podaci['city'].upper()

    

In [16]:
df_banks = DataFrame.from_dict(banks)

df_banks = df_banks.T
df_banks.reset_index(inplace=True)

df_banks.rename(columns={'index': 'bank_name', 'city': 'city_name'}, inplace=True)

In [17]:
df_cities = df_cities[['city_name', 'pk', 'stopa']]

In [18]:
df = DataFrame.merge(df_cities, df_banks, on='city_name', how='right')

df = df[['bank_name', 'oib', 'mib', 'iban', 'address', 'pk']]
df.rename(columns={'pk': 'city', 'oib': 'pk'}, inplace=True)

In [19]:
banks_dict = df.to_dict(orient='index')

In [22]:
bank_name = 'bank_name'
pk = 'pk'
mib = 'mib'
address = 'address'
city = 'city'
iban = 'iban'

In [24]:
j = '['

for data in banks_dict.values():
    j += f'{{"model": "third_parties.bank", "pk": "{data[pk]}", "fields": {{"bank_name": "{data[bank_name]}", "mib": "{data[mib]}", "iban": "{data[iban]}", "address": "{data[address]}", "city": "{data[city]}"}}}},\n'

j = j[:-1] + ']'

In [25]:
banks_json = open('banks_fixtures.json', 'w', encoding='UTF-8')

banks_json.write(j)

banks_json.close()

'[{"model": "third_parties.bank", "pk": "14036333877", fields: {"bank_name": "ADDIKO BANK d.d.", "mib": "1198947", "iban": "HR6025000091000000013", "address": Slavonska avenija 6, "city": "01333"}},\n{"model": "third_parties.bank", "pk": "70663193635", fields: {"bank_name": "AGRAM BANKA d.d. Zagreb", "mib": "80003981", "iban": "HR4424810001011111116", "address": Ulica grada Vukovara 74, "city": "01333"}},\n{"model": "third_parties.bank", "pk": "32247795989", fields: {"bank_name": "CROATIA BANKA d.d.", "mib": "03467988", "iban": "HR0324850031000009027", "address": Roberta Frangeša Mihanovića 9, "city": "01333"}},\n{"model": "third_parties.bank", "pk": "87939104217", fields: {"bank_name": "HRVATSKA POŠTANSKA BANKA d.d. Zagreb", "mib": "3777928", "iban": "HR4623900011070000029", "address": Jurišićeva 4, "city": "01333"}},\n{"model": "third_parties.bank", "pk": "73656725926", fields: {"bank_name": "KENTBANK d.d. Zagreb", "mib": "1263986", "iban": "HR5741240031011111116", "address": Gunduli

In [5]:
from bs4 import BeautifulSoup
from requests import get, post

In [ ]:
from bs4 import BeautifulSoup
from requests import get

page: int = 1

while True:
    query: str = f'https://www.posta.hr/pretrazivanje-postanskih-ureda/263?pojam=&page={page}'

    try:
        postal_codes_page = get(query)
    except IndexError:
        print('kraj')
        return
    else:

        soup = BeautifulSoup(banks_page.content, 'html.parser')

        div = soup.find_all("div", attrs={"class": "vc_tta-panels-container"})